In [185]:
import pandas as pd
import ast
import numpy as np

In [186]:
path = 'comparative.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [187]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [188]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [189]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...","Questions: ""There are 10 apples on an apple tr...","1. The most likely answer to the question ""The...","to the question ""there are 10 apples on an ap..."


In [190]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [191]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [192]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [193]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)

In [194]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)
paper.index = paper.index.str.strip()

In [195]:
df.set_index('premise', inplace=True)

In [196]:
paper['my_label'] = df['LLM_answer']

In [197]:
sum(paper.label == paper.my_label) / len(paper)

0.726516784724361

In [198]:
df.to_csv('comparative - full.csv')

In [199]:
df

,hypothesis,prompt,rationale,split,correct_index,LLM_answer
premise,,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?","[park, coloring book, garden center, math prob...","Questions: ""There are 10 apples on an apple tr...","1. The most likely answer to the question ""The...","to the question ""there are 10 apples on an ap...","{'math problem': 137, 'gravity': 302}",math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?","[bus depot, beach, train station, bridge, bridge]",Questions: A John is a bum. Much like the ste...,2. The most likely answer to the question A Jo...,to the question a john is a bum. much like t...,{'bridge': 148},bridge
"A bad person places little value on being honest, acting without pretense or being what?","[excellent, upright, premium, competent, sincere]",Questions: A bad person places little value on...,3. The most likely answer to the question A ba...,to the question a bad person places little va...,{'sincere': 115},sincere
"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]","Questions: A bald eagle flies over St. Paul, w...",4. The most likely answer to the question A ba...,to the question a bald eagle flies over st. p...,"{'minnesota': 73, 'photograph': 309}",minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?","[yatch, corvette, aircraft carrier, destroyer,...",Questions: A battleship is a powerful vessel. ...,5. The most likely answer to the question A ba...,to the question a battleship is a powerful ve...,{'destroyer': 124},destroyer
...,...,...,...,...,...,...
What happens to a building around half of the time every day?,"[it gets dark, cast shadow, face north, target...",NaN,"3. The most likely answer to the question ""Wha...","to the question ""what happens to a building a...","{'it gets dark': 90, 'cast shadow': 268, 'face...",it gets dark
What is the result of more people getting in line?,"[anxiety, wait turn, late, longer lines, have ...",NaN,"4. The most likely answer to the question ""Wha...","to the question ""what is the result of more p...","{'anxiety': 245, 'wait turn': 405, 'late': 497...",longer lines
"John wanted to hunt blowfish, but he didn't want to leave America. Where might he search?","[tropical sea, south pacific ocean, hawaiian w...",NaN,1. The most likely answer to the question is *...,to the question is **(c) hawaiian waters**. b...,"{'tropical sea': 337, 'south pacific ocean': 3...",hawaiian waters
